In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/physionet-ecg/scg_rhc_f16.parquet


# Run this cell first. It installs a few libs not always preinstalled.
!pip install --quiet neurokit2 xgboost pyarrow tqdm



import pandas as pd

# Path to the parquet file
file_path = "/kaggle/input/physionet-ecg/scg_rhc_f16.parquet"

# Load the parquet file
df = pd.read_parquet(file_path)

# Show dataset shape and first rows
print("Shape of dataset:", df.shape)



# Start

In [2]:
!pip install --quiet neurokit2 xgboost pyarrow tqdm matplotlib seaborn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 708.4/708.4 kB 15.8 MB/s eta 0:00:00


# Load parquet and inspect ECG lead II contents

import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
from tqdm.auto import tqdm
pd.set_option('display.max_columns', 200)

pq_path = "/kaggle/input/physionet-ecg/scg_rhc_f16.parquet"   # your file
df = pd.read_parquet(pq_path)
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
df.head(2)


# Quick diagnostic of ECG_lead_II arrays: lengths, sample stats, plot examples

# examine the type & lengths of ECG_lead_II content
ecg_col = 'ECG_lead_II'
assert ecg_col in df.columns, f"{ecg_col} not present"

lengths = []
mins = []
maxs = []
for i, arr in enumerate(df[ecg_col]):
    # arr is likely a list or numpy array; coerce to numpy
    a = np.asarray(arr, dtype=float)
    lengths.append(a.size)
    mins.append(a.min())
    maxs.append(a.max())
    
lengths = np.array(lengths)
print("ECG_lead_II lengths: min, median, max =", lengths.min(), np.median(lengths), lengths.max())
print("ECG amplitude: min mean, max mean =", np.mean(mins), np.mean(maxs))

# show distribution of lengths
plt.figure(figsize=(6,3))
sns.histplot(lengths, bins=30)
plt.title("Distribution of ECG_lead_II sample counts per row")
plt.xlabel("Number of samples")
plt.show()

# plot first 3 ECG traces (normalized)
plt.figure(figsize=(12,6))
for i in range(3):
    a = np.asarray(df[ecg_col].iloc[i], dtype=float)
    t = np.arange(a.size)
    # normalize for visualization
    a = (a - a.mean()) / (a.std() + 1e-9)
    plt.plot(t, a + i*6, label=f"rec{i}")  # offset vertically
plt.title("First 3 ECG_lead_II traces (standardized, vertically offset)")
plt.xlabel("Sample index")
plt.legend()
plt.show()


# Auto-select plausible sampling rate per recording

import neurokit2 as nk

candidate_fs = [125, 250, 500]
def estimate_fs_for_signal(arr):
    arr = np.asarray(arr, dtype=float)
    best_fs = None
    best_hr_diff = 1e9
    best_hr = None
    for fs in candidate_fs:
        try:
            # use ECG peak detector only (fast)
            _, rpeaks = nk.ecg_peaks(arr, sampling_rate=fs)
            r_inds = rpeaks.get('ECG_R_Peaks', [])
            if len(r_inds) < 3:
                continue
            rr = np.diff(np.array(r_inds)) / fs
            mean_hr = 60.0 / rr.mean()
            # prefer fs where HR falls in plausible range
            diff = 0 if (40 <= mean_hr <= 140) else abs(mean_hr - 80)
            if diff < best_hr_diff:
                best_hr_diff = diff
                best_fs = fs
                best_hr = mean_hr
        except Exception:
            continue
    return best_fs, best_hr

# test on first 30 recordings
estimates = []
for i in range(min(30, len(df))):
    fs, hr = estimate_fs_for_signal(df[ecg_col].iloc[i])
    estimates.append((i, fs, hr))

pd.DataFrame(estimates, columns=['idx','chosen_fs','mean_hr']).head(20)


# Windowing + HRV feature extraction functions

from scipy.signal import butter, filtfilt, iirnotch

def bandpass(sig, fs, low=0.5, high=40, order=4):
    from scipy.signal import butter, filtfilt
    b,a = butter(order, [low/(fs/2), high/(fs/2)], btype='band')
    return filtfilt(b,a,sig)

def notch(sig, fs, freq=50.0, q=30.0):
    from scipy.signal import iirnotch, filtfilt
    b,a = iirnotch(freq/(fs/2), q)
    return filtfilt(b,a,sig)

def extract_hrv_features_from_window(w, fs):
    """Return dict of HRV/time-domain features for window w (1D numpy array)."""
    out = {}
    # basic amplitude check
    out['max_abs'] = float(np.max(np.abs(w)))
    if out['max_abs'] < 1e-6:
        out['sqi'] = 0
        return out
    # filter (safe try)
    try:
        wf = bandpass(w, fs)
        wf = notch(wf, fs, freq=50.0)
    except Exception:
        wf = w.copy()
    # detect R peaks robustly
    try:
        signals, info = nk.ecg_process(wf, sampling_rate=fs)
        rpeaks = info.get("ECG_R_Peaks", [])
        rpeaks = np.array(rpeaks)
    except Exception:
        rpeaks = np.array(nk.ecg_peaks(wf, sampling_rate=fs)[1].get('ECG_R_Peaks', []))
    if len(rpeaks) < 4:
        out['sqi'] = 0
        return out
    rr_s = np.diff(rpeaks) / fs  # in seconds
    rr_ms = rr_s * 1000.0
    out['sqi'] = 1
    out['mean_hr'] = float(60.0 / np.mean(rr_s))
    out['sdnn'] = float(np.std(rr_ms, ddof=1))
    # rmssd
    if len(rr_ms) >= 3:
        diff_rr = np.diff(rr_ms)
        out['rmssd'] = float(np.sqrt(np.mean(diff_rr**2)))
        out['pnn50'] = float(np.mean(np.abs(diff_rr) > 50.0))
    else:
        out['rmssd'] = np.nan
        out['pnn50'] = np.nan
    # sample entropy (RR in ms)
    try:
        out['sampen'] = float(nk.entropy_sample(rr_ms, normalize=True))
    except Exception:
        out['sampen'] = np.nan
    return out


# Build feature windows across recordings and save to CSV

import math, os
features = []
max_recs = 200   # limit for speed; increase if you want more data
win_s = 30
hop_s = 15

for idx, row in tqdm(df.iloc[:max_recs].iterrows(), total=min(max_recs, len(df))):
    arr = np.asarray(row[ecg_col], dtype=float)
    fs, hr = estimate_fs_for_signal(arr)
    if fs is None:
        # if we couldn't detect fs, skip this recording
        continue
    npts = arr.size
    nwin = (npts - int(win_s*fs)) // int(hop_s*fs) + 1
    if nwin <= 0:
        # if recording too short for a single window, skip
        continue
    for start in range(0, npts - int(win_s*fs) + 1, int(hop_s*fs)):
        w = arr[start:start+int(win_s*fs)]
        feats = extract_hrv_features_from_window(w, fs)
        if feats.get('sqi', 0) == 0:
            continue
        rec = {
            'rec_index': int(idx),
            'start_sample': int(start),
            'fs': int(fs),
            'win_s': win_s,
            'n_samples': int(w.size),
        }
        rec.update(feats)
        # optional metadata
        if 'record_name' in df.columns:
            rec['record_name'] = row['record_name']
        features.append(rec)

feats_df = pd.DataFrame(features)
print("Feature windows extracted:", feats_df.shape)
feats_df.head()
feats_df.to_csv('ecg_feature_windows_scg_rhc_sample.csv', index=False)
print("Saved ecg_feature_windows_scg_rhc_sample.csv")


# Quick unsupervised baseline: KMeans clustering + visualization

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# dynamically pick only available numeric HRV columns
candidate_cols = ['mean_hr','sdnn','rmssd','pnn50','sampen']
num_cols = [c for c in candidate_cols if c in feats_df.columns and feats_df[c].notna().sum() > 0]
print("Using feature columns:", num_cols)

# filter rows where at least these columns are non-null
feats_df_clean = feats_df.dropna(subset=num_cols).copy()
print("Remaining rows after dropna:", len(feats_df_clean))

# prepare features
X = feats_df_clean[num_cols].values
scaler = StandardScaler()
Xs = scaler.fit_transform(X)

# PCA for visualization
pca = PCA(n_components=2)
Xp = pca.fit_transform(Xs)

# KMeans clustering
k = 3
km = KMeans(n_clusters=k, random_state=42, n_init=20)
labels = km.fit_predict(Xs)
sil = silhouette_score(Xs, labels)
print("Silhouette score (k=3):", sil)

feats_df_clean['cluster'] = labels

# visualize clusters
plt.figure(figsize=(7,5))
sns.scatterplot(x=Xp[:,0], y=Xp[:,1], hue=labels, palette='tab10', s=40, alpha=0.8)
plt.title("PCA of HRV features colored by KMeans cluster")
plt.show()

# cluster centers in original feature space
centers = scaler.inverse_transform(km.cluster_centers_)
centers_df = pd.DataFrame(centers, columns=num_cols)
print("Cluster centers (HRV features):")
display(centers_df)


# assign ordering: higher sdnn -> lighter state; so depth_score = descending sdnn ranking
centers_df['cluster'] = centers_df.index
order = centers_df.sort_values('sdnn', ascending=False)['cluster'].tolist()
# map cluster -> pseudo depth (0 = light, 2 = deep) - invert if you prefer
cluster_to_depth = {c: i for i,c in enumerate(order)}  
print("cluster_to_depth mapping:", cluster_to_depth)

feats_df_clean['pseudo_depth'] = feats_df_clean['cluster'].map(cluster_to_depth)

# train/test split by rec_index (to avoid leakage across windows)
from sklearn.model_selection import GroupShuffleSplit
groups = feats_df_clean['rec_index'].values
X = feats_df_clean[num_cols].fillna(0.0).values
y = feats_df_clean['pseudo_depth'].values

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups))
X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

import xgboost as xgb
model = xgb.XGBRegressor(n_estimators=100, max_depth=4, learning_rate=0.05, random_state=42)
model.fit(X_train, y_train, eval_set=[(X_test,y_test)], early_stopping_rounds=10, verbose=False)

preds = model.predict(X_test)
from sklearn.metrics import mean_absolute_error, r2_score
print("MAE:", mean_absolute_error(y_test, preds))
print("R2:", r2_score(y_test, preds))

# Save model to file
import joblib
joblib.dump(model, 'xgb_pseudo_depth_model.joblib')
print("Saved xgb_pseudo_depth_model.joblib")


'''import numpy as np
import matplotlib.pyplot as plt

# use the detected column, or fail with clear message
if ecg_col_found is None:
    raise RuntimeError("No ECG column detected in df. Re-run the parquet load cell or check the file.")

# ensure rec_index values are valid indices (we used iloc, so rec_idx must be < len(df))
valid_recs = feats_df_clean['rec_index'].dropna().astype(int).unique()
print("Unique rec_index count in features:", len(valid_recs), " (example first 10):", valid_recs[:10])

for c in sorted(feats_df_clean['cluster'].unique()):
    # choose first window row for this cluster
    row = feats_df_clean[feats_df_clean['cluster']==c].iloc[0]
    rec_idx = int(row['rec_index'])
    start = int(row['start_sample'])
    fs = int(row['fs'])
    n_samples = int(row['n_samples'])

    # defensive checks
    if rec_idx < 0 or rec_idx >= len(df):
        print(f"Skipping cluster {c}: rec_idx {rec_idx} out of range (0..{len(df)-1})")
        continue

    # pull the array safely using iloc and the detected column name
    try:
        arr_raw = df.iloc[rec_idx][ecg_col_found]
    except Exception as e:
        print(f"Skipping cluster {c}: cannot access ECG at rec {rec_idx}: {e}")
        continue

    arr = np.asarray(arr_raw, dtype=float)
    if arr.size == 0:
        print(f"Skipping cluster {c}: ECG array for rec {rec_idx} is empty")
        continue

    # clip end index safely
    end = start + n_samples
    if start >= arr.size:
        print(f"Cluster {c} rec {rec_idx}: start {start} >= arr length {arr.size} -> skipping")
        continue
    if end > arr.size:
        end = arr.size

    w = arr[start:end]
    if w.size == 0:
        print(f"Cluster {c} rec {rec_idx}: extracted window is empty -> skipping")
        continue

    # time axis: if n_samples was for standardized window, scale time proportionally
    win_duration = row.get('win_s', n_samples / fs if fs>0 else 1.0)
    t = np.linspace(0, win_duration * (w.size / max(n_samples,1)), w.size)

    plt.figure(figsize=(9,2.2))
    plt.plot(t, (w - w.mean()) / (w.std()+1e-9))
    plt.title(f"Cluster {c} example window (rec {rec_idx}, start {start}, end {end}, fs {fs})")
    plt.xlabel("seconds")
    plt.tight_layout()
    plt.show()
'''

# FULLL CODDEE

# Full end-to-end Kaggle-ready notebook
# Copy this whole block into a Kaggle Python notebook and run—make sure the dataset path matches.

# ========================================================
# 0) Installs (run once at top)
# ========================================================
!pip install --quiet neurokit2 xgboost joblib seaborn tqdm

# ========================================================
# 1) Imports & constants
# ========================================================
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (9,4)

# Modeling imports
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, mean_absolute_error, r2_score
from sklearn.model_selection import GroupShuffleSplit
import xgboost as xgb
import joblib

# Signal processing
import neurokit2 as nk
from scipy.signal import butter, filtfilt, iirnotch

# ========================================================
# 2) Load parquet and inspect
# ========================================================
PQ_PATH = "/kaggle/input/physionet-ecg/scg_rhc_f16.parquet"  # update if necessary
assert os.path.exists(PQ_PATH), f"Parquet file not found at {PQ_PATH}"
df = pd.read_parquet(PQ_PATH)
print("Loaded parquet:", PQ_PATH)
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())

# find a good ECG column (prefer ECG_lead_II if present)
preferred = ["ECG_lead_II", "ECG_lead_ii", "ECG_lead_II ", "ECG_lead_I", "ecg_lead_ii"]
ecg_col = None
for p in preferred:
    if p in df.columns:
        ecg_col = p
        break
# fallback: first column containing 'ecg'
if ecg_col is None:
    for c in df.columns:
        if 'ecg' in c.lower():
            ecg_col = c
            break

if ecg_col is None:
    raise RuntimeError("No ECG column found in the parquet file. Inspect df.columns.")
print("Using ECG column:", ecg_col)

# ========================================================
# 3) Diagnostics: lengths, amplitude, plot a few traces
# ========================================================
lengths = []
mins = []
maxs = []
for arr in df[ecg_col]:
    a = np.asarray(arr, dtype=float)
    lengths.append(a.size)
    mins.append(a.min())
    maxs.append(a.max())
lengths = np.array(lengths)
print("ECG sample counts per row — min/median/max:", lengths.min(), np.median(lengths), lengths.max())
print("Mean amplitude min/mean, max/mean:", np.mean(mins), np.mean(maxs))

# quick histogram of lengths
plt.figure(figsize=(6,3))
sns.histplot(lengths, bins=30)
plt.title("ECG sample count distribution (per row)")
plt.xlabel("Number of samples")
plt.show()

# plot first 3 traces (standardized)
plt.figure(figsize=(12,4))
for i in range(min(3, len(df))):
    a = np.asarray(df[ecg_col].iloc[i], dtype=float)
    a = (a - a.mean()) / (a.std() + 1e-9)
    plt.plot(a + i*6, label=f"rec{i}")
plt.title("First 3 ECG traces (standardized & vertically offset)")
plt.legend()
plt.show()

# ========================================================
# 4) Helpers: filtering, FS estimation, HRV extraction
# ========================================================
candidate_fs = [125, 250, 500, 360]   # common sampling rates (added 360)
def bandpass(sig, fs, low=0.5, high=40, order=4):
    b, a = butter(order, [low/(fs/2), high/(fs/2)], btype='band')
    return filtfilt(b, a, sig)

def notch(sig, fs, freq=50.0, q=30.0):
    b, a = iirnotch(freq/(fs/2), q)
    return filtfilt(b, a, sig)

def estimate_fs_for_signal(arr, candidates=candidate_fs):
    """Choose fs that yields a physiologically plausible mean HR after R-peak detection."""
    arr = np.asarray(arr, dtype=float)
    best_fs, best_hr, best_score = None, None, 1e9
    for fs in candidates:
        try:
            peaks = nk.ecg_peaks(arr, sampling_rate=fs)[1].get('ECG_R_Peaks', [])
            if len(peaks) < 3:
                continue
            rr = np.diff(np.array(peaks)) / fs
            mean_hr = 60.0 / rr.mean()
            score = 0 if (40 <= mean_hr <= 140) else abs(mean_hr - 80)
            if score < best_score:
                best_score = score
                best_fs = fs
                best_hr = mean_hr
        except Exception:
            continue
    return best_fs, best_hr

def extract_hrv_features_from_window(w, fs):
    """Compute robust HRV time-domain features from one ECG window."""
    out = {}
    w = np.asarray(w, dtype=float)
    if w.size == 0:
        return None
    out['max_abs'] = float(np.max(np.abs(w)))
    if out['max_abs'] < 1e-6:
        out['sqi'] = 0
        return out
    # filter (try/catch)
    try:
        wf = bandpass(w, fs)
        # choose notch frequency by region if you know, default 50
        wf = notch(wf, fs, freq=50.0)
    except Exception:
        wf = w.copy()
    # R-peak detection
    try:
        signals, info = nk.ecg_process(wf, sampling_rate=fs)
        rpeaks = np.array(info.get("ECG_R_Peaks", []))
    except Exception:
        rpeaks = np.array(nk.ecg_peaks(wf, sampling_rate=fs)[1].get('ECG_R_Peaks', []))
    if len(rpeaks) < 4:
        out['sqi'] = 0
        return out
    rr_s = np.diff(rpeaks) / fs
    rr_ms = rr_s * 1000.0
    out['sqi'] = 1
    out['mean_hr'] = float(60.0 / np.mean(rr_s))
    out['sdnn'] = float(np.std(rr_ms, ddof=1))
    if len(rr_ms) >= 3:
        diff_rr = np.diff(rr_ms)
        out['rmssd'] = float(np.sqrt(np.mean(diff_rr**2)))
        out['pnn50'] = float(np.mean(np.abs(diff_rr) > 50.0))
    else:
        out['rmssd'] = np.nan
        out['pnn50'] = np.nan
    # sample entropy often fails with few RR -> skip here (we won't rely on it)
    out['sampen'] = np.nan
    return out

# ========================================================
# 5) Windowing & feature extraction (30s windows, 50% overlap)
# ========================================================
features = []
max_recs = 200  # keep small for fast iteration; increase if you want more data
win_s = 30
hop_s = 15

pbar = tqdm(range(min(max_recs, len(df))), desc="Records")
for idx in pbar:
    arr = np.asarray(df[ecg_col].iloc[idx], dtype=float)
    fs, hr = estimate_fs_for_signal(arr)
    if fs is None:
        # skip if we can't estimate sampling rate
        continue
    npts = arr.size
    if npts < int(win_s * fs):
        continue
    for start in range(0, npts - int(win_s*fs) + 1, int(hop_s*fs)):
        w = arr[start:start + int(win_s*fs)]
        feats = extract_hrv_features_from_window(w, fs)
        if feats is None:
            continue
        if feats.get('sqi', 0) == 0:
            continue
        rec = {
            'rec_index': int(idx),
            'start_sample': int(start),
            'fs': int(fs),
            'win_s': win_s,
            'n_samples': int(w.size)
        }
        rec.update(feats)
        if 'record_name' in df.columns:
            rec['record_name'] = df['record_name'].iloc[idx]
        features.append(rec)

feats_df = pd.DataFrame(features)
print("Extracted feature windows:", feats_df.shape)
display(feats_df.head())
feats_df.to_csv('ecg_feature_windows_scg_rhc_sample.csv', index=False)
print("Saved ecg_feature_windows_scg_rhc_sample.csv")

# ========================================================
# 6) Prepare feature matrix (drop all-NaN cols like sampen)
# ========================================================
# Candidate features: prefer time-domain HRV features we computed
candidate_cols = ['mean_hr', 'sdnn', 'rmssd', 'pnn50', 'sampen']
# pick only columns that exist and have at least one non-null value
num_cols = [c for c in candidate_cols if c in feats_df.columns and feats_df[c].notna().sum() > 0]
print("Using numeric features:", num_cols)

# require at least 1 row
if len(feats_df) == 0 or len(num_cols) == 0:
    raise RuntimeError("No usable feature windows extracted. Re-check preprocessing or increase max_recs.")

# drop rows missing these features and fill any remaining NaNs with median
feats_df_clean = feats_df.dropna(subset=num_cols).copy()
for c in num_cols:
    if feats_df_clean[c].isna().any():
        median = feats_df_clean[c].median()
        feats_df_clean[c].fillna(median, inplace=True)

print("Feature windows after cleaning:", feats_df_clean.shape)

# ========================================================
# 7) Unsupervised clustering -> pseudo labels
# ========================================================
X = feats_df_clean[num_cols].values
scaler = StandardScaler()
Xs = scaler.fit_transform(X)

# PCA for plotting
pca = PCA(n_components=2, random_state=42)
Xp = pca.fit_transform(Xs)

# KMeans clustering (k=3)
k = 3
km = KMeans(n_clusters=k, random_state=42, n_init=20)
labels = km.fit_predict(Xs)
sil = silhouette_score(Xs, labels) if Xs.shape[0] > 1 else float("nan")
print("Silhouette score (k=3):", sil)

feats_df_clean['cluster'] = labels
plt.figure(figsize=(7,5))
sns.scatterplot(x=Xp[:,0], y=Xp[:,1], hue=labels, palette='tab10', s=40, alpha=0.8)
plt.title("PCA of HRV features colored by KMeans cluster")
plt.show()

# cluster centers in original feature space
centers = scaler.inverse_transform(km.cluster_centers_)
centers_df = pd.DataFrame(centers, columns=num_cols)
print("Cluster centers (HRV features):")
display(centers_df)

# ========================================================
# 8) Map clusters -> ordered pseudo-depth (heuristic via sdnn)
# ========================================================
# order clusters by sdnn descending (higher sdnn -> lighter state)
if 'sdnn' in centers_df.columns:
    centers_df['cluster'] = centers_df.index
    order = centers_df.sort_values('sdnn', ascending=False)['cluster'].tolist()
else:
    order = list(range(k))
cluster_to_depth = {c: i for i,c in enumerate(order)}  # 0 = light, larger = deeper (by sdnn)
print("cluster_to_depth mapping:", cluster_to_depth)

feats_df_clean['pseudo_depth'] = feats_df_clean['cluster'].map(cluster_to_depth)

# ========================================================
# 9) Train supervised model (XGBoost) on pseudo labels (group split by rec_index)
# ========================================================
groups = feats_df_clean['rec_index'].values
X_mat = feats_df_clean[num_cols].fillna(0.0).values
y = feats_df_clean['pseudo_depth'].values

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X_mat, y, groups))
X_train, X_test = X_mat[train_idx], X_mat[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

model = xgb.XGBRegressor(n_estimators=200, max_depth=5, learning_rate=0.05, random_state=42)
model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=20, verbose=False)

preds = model.predict(X_test)
print("MAE:", mean_absolute_error(y_test, preds))
print("R2 :", r2_score(y_test, preds))

# save model and scaler
joblib.dump(model, "xgb_pseudo_depth_model.joblib")
joblib.dump(scaler, "feature_scaler.joblib")
print("Saved model and scaler.")

# ========================================================
# 10) Visualizations for demo
# ========================================================
# show example ECG window from each cluster
print("Showing one example ECG window per cluster (if available):")
for c in sorted(feats_df_clean['cluster'].unique()):
    subset = feats_df_clean[feats_df_clean['cluster']==c]
    if subset.shape[0] == 0:
        continue
    row = subset.iloc[0]
    rec_idx = int(row['rec_index'])
    start = int(row['start_sample']); fs = int(row['fs'])
    n_samples = int(row['n_samples'])
    arr_raw = np.asarray(df[ecg_col].iloc[rec_idx], dtype=float)
    end = min(start + n_samples, arr_raw.size)
    w = arr_raw[start:end]
    t = np.linspace(0, row['win_s'], w.size)
    plt.figure(figsize=(9,2))
    plt.plot(t, (w - w.mean())/(w.std()+1e-9))
    plt.title(f"Cluster {c} example (rec {rec_idx}, fs {fs})")
    plt.xlabel("seconds")
    plt.show()

# show cluster timeline for one recording (choose a rec with multiple windows)
example_rec = int(feats_df_clean['rec_index'].mode()[0])
rec_windows = feats_df_clean[feats_df_clean['rec_index']==example_rec].sort_values('start_sample')
if rec_windows.shape[0] > 0:
    times = rec_windows['start_sample'] / rec_windows['fs']
    plt.figure(figsize=(10,2))
    plt.plot(times, rec_windows['pseudo_depth'], marker='o')
    plt.gca().invert_yaxis()  # lower plot = lighter depth (optional)
    plt.title(f"Pseudo-depth timeline for record {example_rec}")
    plt.xlabel("seconds from start")
    plt.ylabel("pseudo_depth (0=light)")
    plt.show()
else:
    print("No windows found for example record.")

# ========================================================
# 11) How to replace pseudo labels with real labels later
# ========================================================
print("""
DONE: pipeline complete.

Notes for next steps:
- When you obtain clinical labels (GCS/BIS) for each window:
  * create feats_df_clean['true_label'] aligned by rec_index & start time,
  * replace y with feats_df_clean['true_label'] and retrain model (same code).
- This notebook currently uses pseudo labels (KMeans on HRV). It's a demo/prototype.
""")


# ========================================================
# 0) Installs (run once at top)
# ========================================================
!pip install --quiet neurokit2 xgboost joblib seaborn tqdm streamlit

# ========================================================
# 1) Imports & constants
# ========================================================
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (9,4)

# Modeling imports
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, mean_absolute_error, r2_score
from sklearn.model_selection import GroupShuffleSplit
import xgboost as xgb
import joblib

# Signal processing
import neurokit2 as nk
from scipy.signal import butter, filtfilt, iirnotch

# ========================================================
# 2) Load parquet and inspect
# ========================================================
PQ_PATH = "/kaggle/input/physionet-ecg/scg_rhc_f16.parquet"
assert os.path.exists(PQ_PATH), f"Parquet file not found at {PQ_PATH}"
df = pd.read_parquet(PQ_PATH)
print("Loaded parquet:", PQ_PATH)
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())

preferred = ["ECG_lead_II", "ECG_lead_ii", "ECG_lead_II ", "ECG_lead_I", "ecg_lead_ii"]
ecg_col = None
for p in preferred:
    if p in df.columns:
        ecg_col = p
        break
if ecg_col is None:
    for c in df.columns:
        if 'ecg' in c.lower():
            ecg_col = c
            break
if ecg_col is None:
    raise RuntimeError("No ECG column found in the parquet file.")
print("Using ECG column:", ecg_col)

# ========================================================
# 3) Quick diagnostics
# ========================================================
lengths = [len(np.asarray(arr, float)) for arr in df[ecg_col]]
print("ECG sample counts per row — min/median/max:",
      np.min(lengths), np.median(lengths), np.max(lengths))

# ========================================================
# 4) Helpers: filtering, FS estimation, HRV extraction
# ========================================================
candidate_fs = [125, 250, 360, 500]

def bandpass(sig, fs, low=0.5, high=40, order=4):
    b, a = butter(order, [low/(fs/2), high/(fs/2)], btype='band')
    return filtfilt(b, a, sig)

def notch(sig, fs, freq=50.0, q=30.0):
    b, a = iirnotch(freq/(fs/2), q)
    return filtfilt(b, a, sig)

def estimate_fs_for_signal(arr, candidates=candidate_fs):
    arr = np.asarray(arr, dtype=float)
    best_fs, best_score = None, 1e9
    for fs in candidates:
        try:
            peaks = nk.ecg_peaks(arr, sampling_rate=fs)[1].get('ECG_R_Peaks', [])
            if len(peaks) < 3:
                continue
            rr = np.diff(peaks) / fs
            mean_hr = 60.0 / rr.mean()
            score = 0 if (40 <= mean_hr <= 140) else abs(mean_hr - 80)
            if score < best_score:
                best_score = score
                best_fs = fs
        except Exception:
            continue
    return best_fs, None

# 🔹 NEW: RR diagnostics
def rr_diagnostics(rr_ms):
    flags = {}
    if len(rr_ms) < 3:
        flags['valid'] = False
        return flags
    mean_rr = np.mean(rr_ms)
    cv_rr = np.std(rr_ms) / mean_rr if mean_rr > 0 else np.inf
    flags['valid'] = True
    flags['mean_rr_ms'] = mean_rr
    flags['cv_rr'] = cv_rr
    flags['too_fast'] = mean_rr < 300   # HR > 200 bpm
    flags['too_slow'] = mean_rr > 2000  # HR < 30 bpm
    flags['irregular'] = cv_rr > 0.3
    return flags

# 🔹 UPDATED: feature extraction with LF/HF + RR flags
def extract_hrv_features_from_window(w, fs):
    out = {}
    w = np.asarray(w, dtype=float)
    if w.size == 0:
        return None
    out['max_abs'] = float(np.max(np.abs(w)))
    if out['max_abs'] < 1e-6:
        out['sqi'] = 0
        return out
    try:
        wf = bandpass(w, fs)
        wf = notch(wf, fs, freq=50.0)
    except Exception:
        wf = w.copy()
    try:
        signals, info = nk.ecg_process(wf, sampling_rate=fs)
        rpeaks = np.array(info.get("ECG_R_Peaks", []))
    except Exception:
        rpeaks = np.array(nk.ecg_peaks(wf, sampling_rate=fs)[1].get('ECG_R_Peaks', []))
    if len(rpeaks) < 4:
        out['sqi'] = 0
        return out
    rr_s = np.diff(rpeaks) / fs
    rr_ms = rr_s * 1000.0
    out['sqi'] = 1
    out['mean_hr'] = float(60.0 / np.mean(rr_s))
    out['sdnn'] = float(np.std(rr_ms, ddof=1))
    if len(rr_ms) >= 3:
        diff_rr = np.diff(rr_ms)
        out['rmssd'] = float(np.sqrt(np.mean(diff_rr**2)))
        out['pnn50'] = float(np.mean(np.abs(diff_rr) > 50.0))
    else:
        out['rmssd'] = np.nan
        out['pnn50'] = np.nan
    # 🔹 LF/HF ratio
    try:
        hrv_freq = nk.hrv_frequency(rpeaks, sampling_rate=fs, show=False)
        out['lf_hf'] = float(hrv_freq['HRV_LFHF'].iloc[0])
    except Exception:
        out['lf_hf'] = np.nan
    # 🔹 RR flags
    flags = rr_diagnostics(rr_ms)
    out['rr_valid'] = int(flags.get('valid', 0))
    out['rr_irregular'] = int(flags.get('irregular', 0))
    return out

# ========================================================
# 5) Windowing & feature extraction
# ========================================================
features = []
max_recs = 200
win_s = 30
hop_s = 15

for idx in tqdm(range(min(max_recs, len(df))), desc="Records"):
    arr = np.asarray(df[ecg_col].iloc[idx], dtype=float)
    fs, _ = estimate_fs_for_signal(arr)
    if fs is None:
        continue
    npts = arr.size
    if npts < int(win_s * fs):
        continue
    for start in range(0, npts - int(win_s*fs) + 1, int(hop_s*fs)):
        w = arr[start:start + int(win_s*fs)]
        feats = extract_hrv_features_from_window(w, fs)
        if feats is None or feats.get('sqi', 0) == 0:
            continue
        rec = {
            'rec_index': int(idx),
            'start_sample': int(start),
            'fs': int(fs),
            'win_s': win_s,
            'n_samples': int(w.size)
        }
        rec.update(feats)
        if 'record_name' in df.columns:
            rec['record_name'] = df['record_name'].iloc[idx]
        features.append(rec)

feats_df = pd.DataFrame(features)
print("Extracted feature windows:", feats_df.shape)
feats_df.to_csv('ecg_feature_windows_scg_rhc_sample.csv', index=False)

# ========================================================
# 6) Feature prep
# ========================================================
candidate_cols = ['mean_hr', 'sdnn', 'rmssd', 'pnn50', 'lf_hf']
num_cols = [c for c in candidate_cols if c in feats_df.columns and feats_df[c].notna().sum() > 0]
print("Using numeric features:", num_cols)
feats_df_clean = feats_df.dropna(subset=num_cols).copy()
for c in num_cols:
    if feats_df_clean[c].isna().any():
        feats_df_clean[c].fillna(feats_df_clean[c].median(), inplace=True)

# ========================================================
# 7) Clustering
# ========================================================
X = feats_df_clean[num_cols].values
scaler = StandardScaler()
Xs = scaler.fit_transform(X)
k = 3
km = KMeans(n_clusters=k, random_state=42, n_init=20)
labels = km.fit_predict(Xs)
sil = silhouette_score(Xs, labels)
print("Silhouette score (k=3):", sil)
feats_df_clean['cluster'] = labels
centers_df = pd.DataFrame(scaler.inverse_transform(km.cluster_centers_), columns=num_cols)
print("Cluster centers:\n", centers_df)

# ========================================================
# 8) Pseudo-depth mapping
# ========================================================
order = centers_df.sort_values('sdnn', ascending=False).index.tolist()
cluster_to_depth = {c: i for i, c in enumerate(order)}
feats_df_clean['pseudo_depth'] = feats_df_clean['cluster'].map(cluster_to_depth)

# ========================================================
# 9) Train XGBoost regressor
# ========================================================
groups = feats_df_clean['rec_index'].values
X_mat = feats_df_clean[num_cols].values
y = feats_df_clean['pseudo_depth'].values
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X_mat, y, groups))
X_train, X_test = X_mat[train_idx], X_mat[test_idx]
y_train, y_test = y[train_idx], y[test_idx]
model = xgb.XGBRegressor(n_estimators=200, max_depth=5, learning_rate=0.05, random_state=42)
model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=20, verbose=False)
preds = model.predict(X_test)
print("MAE:", mean_absolute_error(y_test, preds))
print("R2 :", r2_score(y_test, preds))


# ========================================================
# 9b) Save Model and Scaler
# ========================================================
import joblib
import os

# Create an "outputs" folder (Kaggle recommends this)
os.makedirs("outputs", exist_ok=True)

# Save trained model
model_path = "outputs/ecg_sleep_model.pkl"
joblib.dump(model, model_path)

# Save scaler
scaler_path = "outputs/scaler.pkl"
joblib.dump(scaler, scaler_path)

print(f"✅ Model saved at: {model_path}")
print(f"✅ Scaler saved at: {scaler_path}")

# Check files
print("Files in outputs folder:", os.listdir("outputs"))


# ========================================================
# 10) Streamlit demo (save as app.py in same dir)
# ========================================================
demo_code = r'''
# app.py - Streamlit app for ECG -> pseudo-depth timeline (copy/paste entire file)

import streamlit as st
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import neurokit2 as nk
import os

st.set_page_config(layout="wide", page_title="UPACS — ECG pseudo-depth demo")

# ------------------------
# Helpers
# ------------------------
@st.cache_resource
def load_artifacts(model_path="ecg_sleep_model.pkl", scaler_path="scaler.pkl"):
    """Load model & scaler; return (model, scaler) or (None,None) with error msg."""
    try:
        model = joblib.load(model_path)
        scaler = joblib.load(scaler_path)
        return model, scaler, None
    except Exception as e:
        return None, None, str(e)

def infer_sampling_rate_from_timecol(time_arr):
    t = np.asarray(time_arr, dtype=float)
    if t.size < 3: 
        return None
    dt = np.diff(t)
    median_dt = np.median(dt)
    if median_dt <= 0:
        return None
    fs = int(round(1.0 / median_dt))
    return fs

def compute_hrv_features(ecg_signal, fs):
    """
    Compute the HRV time-domain features used by the model:
      mean_hr (bpm), sdnn (ms), rmssd (ms), pnn50 (fraction)
    Return dict or None if detection failed.
    """
    sig = np.asarray(ecg_signal, dtype=float)
    if sig.size < 10:
        return None
    # Clean ECG (NeuroKit2)
    try:
        cleaned = nk.ecg_clean(sig, sampling_rate=fs)
        signals, info = nk.ecg_process(cleaned, sampling_rate=fs)
        rpeaks = np.array(info.get("ECG_R_Peaks", []))
    except Exception:
        # fallback simple peak detection
        rpeaks = np.array(nk.ecg_peaks(sig, sampling_rate=fs)[1].get("ECG_R_Peaks", []))
    if rpeaks.size < 4:
        return None
    rr_s = np.diff(rpeaks) / fs
    if rr_s.size < 2:
        return None
    rr_ms = rr_s * 1000.0
    mean_hr = float(60.0 / np.mean(rr_s))
    sdnn = float(np.std(rr_ms, ddof=1))
    diff_rr = np.diff(rr_ms)
    rmssd = float(np.sqrt(np.mean(diff_rr**2))) if diff_rr.size > 0 else np.nan
    pnn50 = float(np.mean(np.abs(diff_rr) > 50.0)) if diff_rr.size > 0 else np.nan
    return {"mean_hr": mean_hr, "sdnn": sdnn, "rmssd": rmssd, "pnn50": pnn50}

# ------------------------
# Load model + scaler
# ------------------------
model, scaler, load_err = load_artifacts()
if load_err:
    st.warning("Could not load model/scaler automatically. Make sure files exist in the app folder:")
    st.info("Looking for: ecg_sleep_model.pkl and scaler.pkl")
    st.error(load_err)
    # still let user upload ECG to inspect waveform even without model
else:
    st.success("Model and scaler loaded successfully.")

# Determine features order expected by scaler/model
# Preferred: use model.feature_names_in_ if available; otherwise assume the trained HRV set
default_feature_order = ["mean_hr", "sdnn", "rmssd", "pnn50"]

if model is not None and hasattr(model, "feature_names_in_"):
    expected_features = list(model.feature_names_in_)
else:
    expected_features = default_feature_order.copy()

# Ensure scaler.feature count matches expected features length (fallback if mismatch)
if scaler is not None:
    n_expected = scaler.n_features_in_
    if n_expected != len(expected_features):
        # take first n_expected from default (most likely case)
        expected_features = default_feature_order[:n_expected]

st.title("UPACS — ECG → Pseudo-Depth Demo")
st.write("Upload an ECG CSV (columns: `ecg` and optional `time`) or use sample ECG to test.")

# ------------------------
# File uploader or sample button
# ------------------------
col1, col2 = st.columns([3,1])
with col2:
    use_sample = st.button("Use sample synthetic ECG (10s)")

uploaded_file = st.file_uploader("Upload ECG CSV file (columns: 'ecg' and optional 'time')", type=["csv"])

# If no file uploaded and user clicks sample, create sample
if uploaded_file is None and use_sample:
    # Generate synthetic ECG-like signal (demo only)
    n_samples = 2000
    time = np.linspace(0, 20, n_samples)  # 20 seconds
    ecg_signal = 0.6*np.sin(1.7*np.pi*time) + 0.3*np.sin(3.5*np.pi*time) + 0.05*np.random.randn(n_samples)
    df = pd.DataFrame({"time": time, "ecg": ecg_signal})
    st.success("Loaded sample synthetic ECG (20 s).")
elif uploaded_file is not None:
    try:
        df = pd.read_csv(uploaded_file)
    except Exception as e:
        st.error(f"Failed to read uploaded file: {e}")
        st.stop()
else:
    st.info("Upload a CSV or click 'Use sample' to proceed.")
    st.stop()

# show preview
st.subheader("Uploaded ECG preview")
st.dataframe(df.head(10))

# ------------------------
# Determine ECG column and sampling rate
# ------------------------
# choose ecg column (case-insensitive)
if "ecg" in df.columns:
    ecg_col = "ecg"
else:
    # try other likely names
    candidates = [c for c in df.columns if "ecg" in c.lower() or "signal" in c.lower()]
    if len(candidates) > 0:
        ecg_col = candidates[0]
    else:
        # fallback to numeric column with most rows
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        if len(numeric_cols) == 0:
            st.error("No numeric columns found in uploaded file. Provide a CSV with ECG samples in a column named 'ecg'.")
            st.stop()
        ecg_col = numeric_cols[0]
st.write(f"Using ECG column: `{ecg_col}`")

# infer sampling rate if time column exists
fs = None
if "time" in df.columns:
    fs = infer_sampling_rate_from_timecol(df["time"].values)
    if fs is not None:
        st.write(f"Inferred sampling rate from `time` column: {fs} Hz")
    else:
        st.info("Could not infer sampling rate from `time` column.")
# interactive override
fs_input = st.number_input("Sampling rate (Hz) — if not inferred, set here", min_value=10, max_value=2000, value=fs or 250)
fs = int(fs_input)

# ------------------------
# User options for windowing
# ------------------------
st.sidebar.header("Windowing options")
win_s = st.sidebar.number_input("Window length (seconds)", min_value=5, max_value=120, value=30, step=5)
hop_s = st.sidebar.number_input("Hop length (seconds)", min_value=1, max_value=60, value=5, step=1)

win_samples = int(win_s * fs)
hop_samples = int(hop_s * fs)

ecg_array = np.asarray(df[ecg_col].values, dtype=float)
npts = ecg_array.size
total_time = npts / fs

st.write(f"Signal length: {npts} samples ({total_time:.1f} s) — window: {win_s}s hop: {hop_s}s")

# ------------------------
# Sliding window predictions
# ------------------------
starts = list(range(0, max(1, npts - win_samples + 1), hop_samples))
times = []
preds = []
valid_windows = 0

for start in starts:
    w = ecg_array[start:start + win_samples]
    feats = compute_hrv_features(w, fs)
    if feats is None:
        continue
    # create feature vector in expected order
    X_vec = [feats.get(f, 0.0) for f in expected_features]
    try:
        X_scaled = scaler.transform([X_vec]) if scaler is not None else np.array([X_vec])
        if model is not None:
            pred = model.predict(X_scaled)[0]
        else:
            pred = X_vec[0]  # fallback show HR if no model
    except Exception as e:
        st.warning(f"Scaling/prediction error for window at {start/fs:.1f}s: {e}")
        continue
    preds.append(pred)
    times.append((start + win_samples/2) / fs)  # use window center time
    valid_windows += 1

st.write(f"Valid windows (with detectable R-peaks): {valid_windows} / {len(starts)}")

if len(preds) == 0:
    st.warning("No valid windows found (R-peaks not detected). Try increasing window length or checking sampling rate.")
# ------------------------
# Plotting
# ------------------------
fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True)
# (1) raw ECG (downsample for display if long)
display_len = min(npts, 5000)
step = max(1, npts // display_len)
axes[0].plot(np.arange(0, npts, step)/fs, ecg_array[::step], linewidth=0.7)
axes[0].set_ylabel("ECG amplitude")
axes[0].set_title("Raw ECG signal")

# (2) pseudo-depth timeline / predictions
if len(preds) > 0:
    axes[1].plot(times, preds, marker='o', linestyle='-')
    axes[1].set_ylabel("Pseudo-depth (model output)")
    axes[1].set_xlabel("Time (s)")
    axes[1].set_title("Pseudo-depth timeline (per window center)")
else:
    axes[1].text(0.5, 0.5, "No predictions — no valid windows", ha='center', va='center', fontsize=12)
    axes[1].set_xlabel("Time (s)")

st.pyplot(fig)

# ------------------------
# Show a representative table of features for first few valid windows
# ------------------------
if valid_windows > 0:
    feat_rows = []
    for start in starts:
        w = ecg_array[start:start + win_samples]
        feats = compute_hrv_features(w, fs)
        if feats is None:
            continue
        row = {"start_s": start / fs}
        for k, v in feats.items():
            row[k] = v
        feat_rows.append(row)
        if len(feat_rows) >= 10:
            break
    st.subheader("Example window HRV features (first valid windows)")
    st.dataframe(pd.DataFrame(feat_rows).reset_index(drop=True)) '''

with open("app.py", "w") as f:
    f.write(demo_code)

print("✅ Notebook complete. Run `streamlit run app.py` in a terminal to launch the demo.")


# ========================================================
# 0) Installs
# ========================================================
!pip install --quiet neurokit2 xgboost joblib seaborn tqdm streamlit torch torchvision torchaudio

# ========================================================
# 1) Imports & constants
# ========================================================
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (9,4)

# ML + preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, mean_absolute_error, r2_score
from sklearn.model_selection import GroupShuffleSplit
import xgboost as xgb
import joblib

# Signal processing
import neurokit2 as nk
from scipy.signal import butter, filtfilt, iirnotch

# Deep learning
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

# ========================================================
# 2) Load parquet and pick ECG lead
# ========================================================
PQ_PATH = "/kaggle/input/physionet-ecg/scg_rhc_f16.parquet"
df = pd.read_parquet(PQ_PATH)
print("Loaded parquet:", PQ_PATH, "Shape:", df.shape)

preferred = ["ECG_lead_II", "ECG_lead_ii", "ECG_lead_I"]
ecg_col = None
for p in preferred:
    if p in df.columns:
        ecg_col = p
        break
if ecg_col is None:
    for c in df.columns:
        if "ecg" in c.lower():
            ecg_col = c
            break
if ecg_col is None:
    raise RuntimeError("No ECG column found!")
print("Using ECG column:", ecg_col)

# ========================================================
# 3) Helpers (filtering, HRV)
# ========================================================
def bandpass(sig, fs, low=0.5, high=40, order=4):
    b, a = butter(order, [low/(fs/2), high/(fs/2)], btype="band")
    return filtfilt(b, a, sig)

def notch(sig, fs, freq=50.0, q=30.0):
    b, a = iirnotch(freq/(fs/2), q)
    return filtfilt(b, a, sig)

def estimate_fs_for_signal(arr, candidates=[125,250,360,500]):
    best_fs, best_score = None, 1e9
    for fs in candidates:
        try:
            peaks = nk.ecg_peaks(arr, sampling_rate=fs)[1].get("ECG_R_Peaks", [])
            if len(peaks) < 3: continue
            rr = np.diff(peaks)/fs
            mean_hr = 60.0/rr.mean()
            score = 0 if (40<=mean_hr<=140) else abs(mean_hr-80)
            if score < best_score:
                best_score, best_fs = score, fs
        except: continue
    return best_fs, None

def extract_hrv_features_from_window(w, fs):
    out = {}
    try:
        wf = bandpass(w, fs)
        wf = notch(wf, fs, 50.0)
    except: wf = w
    try:
        signals, info = nk.ecg_process(wf, sampling_rate=fs)
        rpeaks = np.array(info["ECG_R_Peaks"])
    except: return None
    if len(rpeaks)<4: return None
    rr_s = np.diff(rpeaks)/fs
    rr_ms = rr_s*1000
    out["mean_hr"] = 60.0/rr_s.mean()
    out["sdnn"] = np.std(rr_ms, ddof=1)
    diff_rr = np.diff(rr_ms)
    out["rmssd"] = np.sqrt(np.mean(diff_rr**2))
    out["pnn50"] = np.mean(np.abs(diff_rr)>50.0)
    return out

# ========================================================
# 4) Windowing & feature extraction
# ========================================================
features, raw_windows = [], []
max_recs = 200
win_s, hop_s = 30, 15

lengths = [len(np.asarray(arr,float)) for arr in df[ecg_col]]
print("ECG lengths per row — min/median/max:", np.min(lengths), np.median(lengths), np.max(lengths))

for idx in tqdm(range(min(max_recs,len(df))), desc="Records"):
    arr = np.asarray(df[ecg_col].iloc[idx], float)
    fs,_ = estimate_fs_for_signal(arr)
    if fs is None: continue
    npts = arr.size
    if npts < int(win_s*fs): continue
    for start in range(0, npts-int(win_s*fs)+1, int(hop_s*fs)):
        w = arr[start:start+int(win_s*fs)]
        feats = extract_hrv_features_from_window(w, fs)
        if feats is None: continue
        rec = {"rec_index": idx, "fs": fs, "n_samples": len(w)}
        rec.update(feats)
        features.append(rec)
        raw_windows.append(w)

feats_df = pd.DataFrame(features)
print("Extracted windows:", feats_df.shape)

# ========================================================
# 5) Clustering → pseudo labels
# ========================================================
num_cols = ["mean_hr","sdnn","rmssd","pnn50"]
feats_df = feats_df.dropna(subset=num_cols)
raw_windows = [raw_windows[i] for i in feats_df.index]

scaler = StandardScaler()
Xs = scaler.fit_transform(feats_df[num_cols])

k=3
km = KMeans(n_clusters=k, random_state=SEED, n_init=20)
labels = km.fit_predict(Xs)
feats_df["cluster"] = labels
centers = pd.DataFrame(scaler.inverse_transform(km.cluster_centers_),columns=num_cols)
order = centers.sort_values("sdnn", ascending=False).index
mapping = {c:i for i,c in enumerate(order)}
feats_df["pseudo_depth"] = feats_df["cluster"].map(mapping)

# ========================================================
# 6) Baseline XGB
# ========================================================
groups = feats_df["rec_index"].values
X = feats_df[num_cols].values
y = feats_df["pseudo_depth"].values
from sklearn.model_selection import GroupShuffleSplit
train_idx,test_idx = next(GroupShuffleSplit(n_splits=1,test_size=0.2,random_state=SEED).split(X,y,groups))
X_train,X_test,y_train,y_test = X[train_idx],X[test_idx],y[train_idx],y[test_idx]

xgb_model = xgb.XGBRegressor(n_estimators=300, max_depth=5, learning_rate=0.05)
xgb_model.fit(X_train,y_train,eval_set=[(X_test,y_test)],verbose=False)
print("[XGB] MAE:",mean_absolute_error(y_test,xgb_model.predict(X_test)))

# ========================================================
# 7) Dataset with fixed-length signals
# ========================================================
def fix_length(sig, target_len):
    sig = np.asarray(sig,float)
    if len(sig) > target_len: 
        return sig[:target_len]
    elif len(sig) < target_len:
        return np.pad(sig,(0,target_len-len(sig)),"constant")
    return sig

class ECGHRVDataset(Dataset):
    def __init__(self, wins, hrv, labels, target_len):
        self.wins, self.hrv, self.y = wins, hrv, labels
        self.target_len = target_len
    def __len__(self): return len(self.y)
    def __getitem__(self, idx):
        sig = fix_length(self.wins[idx], self.target_len)
        sig = (sig - sig.mean())/(sig.std()+1e-8)
        sig = torch.from_numpy(sig.astype(np.float32)).unsqueeze(0) # [1,T]
        hrv = torch.from_numpy(self.hrv[idx].astype(np.float32))
        y = torch.tensor(self.y[idx],dtype=torch.float32)
        return sig,hrv,y

max_len = max(len(w) for w in raw_windows)  # unify length
train_ds = ECGHRVDataset([raw_windows[i] for i in train_idx],X[train_idx],y[train_idx],max_len)
test_ds  = ECGHRVDataset([raw_windows[i] for i in test_idx],X[test_idx],y[test_idx],max_len)
train_loader = DataLoader(train_ds,batch_size=32,shuffle=True)
test_loader  = DataLoader(test_ds,batch_size=32,shuffle=False)

# ========================================================
# 8) CNN-LSTM Fusion model
# ========================================================
class CNNLSTM_Fusion(nn.Module):
    def __init__(self, hrv_dim, lstm_hidden=64, emb_dim=64):
        super().__init__()
        self.conv1 = nn.Conv1d(1,16,7,padding=3)
        self.pool = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(16,32,5,padding=2)
        self.conv3 = nn.Conv1d(32,64,5,padding=2)
        self.lstm = nn.LSTM(64,lstm_hidden,batch_first=True,bidirectional=True)
        self.emb = nn.Linear(2*lstm_hidden, emb_dim)
        self.fusion = nn.Sequential(
            nn.Linear(emb_dim+hrv_dim,64), nn.ReLU(), nn.Linear(64,1)
        )
    def forward(self,x_sig,x_hrv):
        x = self.pool(torch.relu(self.conv1(x_sig)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.permute(0,2,1)
        out,_ = self.lstm(x)
        last = out[:,-1,:]
        emb = torch.relu(self.emb(last))
        z = torch.cat([emb,x_hrv],dim=1)
        y = self.fusion(z).squeeze(1)
        return y

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CNNLSTM_Fusion(hrv_dim=X.shape[1]).to(device)

# ========================================================
# 9) Train loop
# ========================================================
opt = torch.optim.Adam(model.parameters(),1e-3)
loss_fn = nn.MSELoss()

for ep in range(5): # try 20+
    model.train()
    for sig,hrv,yb in train_loader:
        sig,hrv,yb = sig.to(device),hrv.to(device),yb.to(device)
        opt.zero_grad()
        pred = model(sig,hrv)
        loss = loss_fn(pred,yb)
        loss.backward(); opt.step()
    print(f"Epoch {ep} loss {loss.item():.4f}")

# ========================================================
# 10) Evaluate
# ========================================================
model.eval()
preds,truth = [],[]
with torch.no_grad():
    for sig,hrv,yb in test_loader:
        sig,hrv = sig.to(device),hrv.to(device)
        p = model(sig,hrv)
        preds.append(p.cpu().numpy()); truth.append(yb.numpy())
preds = np.concatenate(preds); truth = np.concatenate(truth)
print("[DL] MAE:",mean_absolute_error(truth,preds),"R2:",r2_score(truth,preds))


In [3]:
!pip install pyhrv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.5/231.5 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.0/158.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.7/225.7 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.8 MB/s eta 0:00:00
  Created wheel for spectrum: filename=spectrum-0.9.0-cp311-cp311-linux_x86_64.whl size=236749 sha256=e2be71fdbc48a031204efb01f0142e5ad67175cfd1569598fefcde882b59780c
  Stored in directory: /root/.cache/pip/wheels/c0/9c/de/eb558fbd03ea1540d3c908f23681f57f9d9e8c2a5cd08d6f42
Successfully built spectrum


In [4]:
# ===============================
# Install dependencies (run once)
# ===============================
# (Uncomment if you need to install)
# !pip install --quiet neurokit2 xgboost joblib scikit-learn scipy tqdm

# ===============================
# Full, robust HRV -> XGBoost pipeline
# ===============================
import os
import math
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import neurokit2 as nk
from scipy.signal import resample, welch, butter, filtfilt, iirnotch

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupShuffleSplit, GroupKFold
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.metrics import silhouette_score

import xgboost as xgb
import joblib

# ---------- reproducibility ----------
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

# ---------- paths & params ----------
PQ_PATH = "/kaggle/input/physionet-ecg/scg_rhc_f16.parquet"
OUT_DIR = "outputs"
os.makedirs(OUT_DIR, exist_ok=True)

MAX_RECS = 374         # set how many records to use (use 374 to use all)
WIN_S = 30             # window length in seconds
HOP_S = 15             # hop in seconds
CANDIDATE_FS = [125, 250, 360, 500]
TARGET_FS = 250        # not strictly required for XGB, but useful if you also want fixed-length windows

print("Loading parquet:", PQ_PATH)
df = pd.read_parquet(PQ_PATH)
print("Loaded parquet:", PQ_PATH, "shape:", df.shape)

# choose ECG column
preferred = ["ECG_lead_II", "ECG_lead_ii", "ECG_lead_II ", "ECG_lead_I", "ecg_lead_ii"]
ecg_col = None
for p in preferred:
    if p in df.columns:
        ecg_col = p
        break
if ecg_col is None:
    for c in df.columns:
        if "ecg" in c.lower():
            ecg_col = c
            break
if ecg_col is None:
    raise RuntimeError("No ECG column found in parquet.")
print("Using ECG column:", ecg_col)

# ---------------------------
# helper: filters & fs estimation
# ---------------------------
def bandpass(sig, fs, low=0.5, high=40, order=4):
    b, a = butter(order, [low/(fs/2), high/(fs/2)], btype='band')
    return filtfilt(b, a, sig)

def notch(sig, fs, freq=50.0, q=30.0):
    b, a = iirnotch(freq/(fs/2), q)
    return filtfilt(b, a, sig)

def estimate_fs_for_signal(arr, candidates=CANDIDATE_FS):
    arr = np.asarray(arr, dtype=float)
    best_fs = None
    best_score = 1e9
    for fs in candidates:
        try:
            peaks_info = nk.ecg_peaks(arr, sampling_rate=fs)[1]
            peaks = peaks_info.get("ECG_R_Peaks", [])
            if len(peaks) < 3:
                continue
            rr = np.diff(peaks) / fs
            mean_hr = 60.0 / rr.mean()
            score = 0 if (40 <= mean_hr <= 140) else abs(mean_hr - 80)
            if score < best_score:
                best_score = score
                best_fs = fs
        except Exception:
            continue
    return best_fs

# ---------------------------
# robust HRV feature extraction per window
# returns dict or None
# ---------------------------
def extract_hrv_features_from_window(w, fs):
    w = np.asarray(w, dtype=float)
    if w.size == 0 or np.max(np.abs(w)) < 1e-6:
        return None
    # try clean/filter
    try:
        wf = bandpass(w, fs)
        wf = notch(wf, fs, freq=50.0)
    except Exception:
        wf = w.copy()
    # R-peak detection
    try:
        signals, info = nk.ecg_process(wf, sampling_rate=fs)
        rpeaks = np.array(info.get("ECG_R_Peaks", []))
    except Exception:
        # fallback
        rpeaks = np.array(nk.ecg_peaks(wf, sampling_rate=fs)[1].get("ECG_R_Peaks", []))
    if len(rpeaks) < 4:
        return None
    rr_s = np.diff(rpeaks) / fs            # in seconds
    if rr_s.size < 2:
        return None
    rr_ms = rr_s * 1000.0                  # ms

    out = {}
    out['mean_hr'] = float(60.0 / rr_s.mean())
    out['sdnn'] = float(np.std(rr_ms, ddof=1))
    # RMSSD & pNN50
    if rr_ms.size >= 2:
        diff_rr = np.diff(rr_ms)
        out['rmssd'] = float(np.sqrt(np.mean(diff_rr**2)))
        out['pnn50'] = float(np.mean(np.abs(diff_rr) > 50.0))
    else:
        out['rmssd'] = np.nan
        out['pnn50'] = np.nan
    # Poincare SD1 / SD2
    if rr_ms.size >= 2:
        sd1 = np.sqrt(0.5) * np.std(np.diff(rr_ms))
        sd1sq = sd1**2
        sdnn = out['sdnn']
        sd2sq = max(0.0, 2.0 * (sdnn**2) - sd1sq)
        sd2 = math.sqrt(sd2sq)
        out['sd1'] = float(sd1)
        out['sd2'] = float(sd2)
    else:
        out['sd1'] = np.nan
        out['sd2'] = np.nan

    # Frequency domain via resampled tachogram -> Welch
    try:
        # times corresponding to each rr interval (use time at the end of each RR)
        times = rpeaks[1:] / float(fs)
        if len(times) >= 4:
            fs_interp = 4.0
            # create interp grid (ensure there is at least a few points)
            t0, t1 = times[0], times[-1]
            if t1 - t0 < 1.0:
                # fallback to small uniform grid
                t_interp = np.linspace(t0, t1, max(4, int((t1 - t0) * fs_interp)))
            else:
                t_interp = np.arange(t0, t1, 1.0 / fs_interp)
                if len(t_interp) < 4:
                    t_interp = np.linspace(t0, t1, max(4, int((t1 - t0) * fs_interp)))
            rr_interp = np.interp(t_interp, times, rr_ms)
            rr_interp = rr_interp - np.mean(rr_interp)
            nperseg = min(256, len(rr_interp))
            fxx, pxx = welch(rr_interp, fs=fs_interp, nperseg=nperseg)
            # LF: 0.04-0.15 Hz, HF: 0.15-0.4 Hz
            lf_mask = (fxx >= 0.04) & (fxx < 0.15)
            hf_mask = (fxx >= 0.15) & (fxx < 0.4)
            lf = float(np.trapz(pxx[lf_mask], fxx[lf_mask])) if lf_mask.any() else 0.0
            hf = float(np.trapz(pxx[hf_mask], fxx[hf_mask])) if hf_mask.any() else 0.0
            total_mask = (fxx >= 0.003) & (fxx < 0.4)
            total_power = float(np.trapz(pxx[total_mask], fxx[total_mask])) if total_mask.any() else (lf + hf)
            out['lf'] = lf
            out['hf'] = hf
            out['lf_hf'] = float(lf / hf) if (hf is not None and hf > 0) else np.nan
            out['total_power'] = total_power
        else:
            out['lf'] = np.nan; out['hf'] = np.nan; out['lf_hf'] = np.nan; out['total_power'] = np.nan
    except Exception:
        out['lf'] = np.nan; out['hf'] = np.nan; out['lf_hf'] = np.nan; out['total_power'] = np.nan

    return out

# ---------------------------
# 4) sliding windows + feature extraction
# ---------------------------
features = []
raw_windows = []
max_recs = min(MAX_RECS, len(df))
lengths = [len(np.asarray(a, float)) for a in df[ecg_col]]
print("ECG sample counts per row — min/median/max:",
      int(np.min(lengths)), int(np.median(lengths)), int(np.max(lengths)))

for idx in tqdm(range(max_recs), desc="Records"):
    arr = np.asarray(df[ecg_col].iloc[idx], dtype=float)
    fs = estimate_fs_for_signal(arr)
    if fs is None:
        # skip unusable record
        continue
    npts = arr.size
    win_pts = int(WIN_S * fs)
    hop_pts = int(HOP_S * fs)
    if npts < win_pts:
        continue
    for start in range(0, npts - win_pts + 1, hop_pts):
        w = arr[start:start + win_pts]
        feats = extract_hrv_features_from_window(w, fs)
        if feats is None:
            continue
        rec = {"rec_index": int(idx), "start_sample": int(start), "fs": int(fs), "n_samples": int(len(w))}
        rec.update(feats)
        features.append(rec)
        # store optionally the resampled window if you want DL later
        raw_windows.append(resample(w, int(round(len(w) * (TARGET_FS / fs)))))

feats_df = pd.DataFrame(features)
print("Extracted feature windows:", feats_df.shape)
if feats_df.shape[0] == 0:
    raise RuntimeError("No valid windows found; check ECG data and sampling rate detection.")

# ---------------------------
# 5) Feature cleanup & selected feature columns
# ---------------------------
candidate_cols = ['mean_hr', 'sdnn', 'rmssd', 'pnn50', 'sd1', 'sd2', 'lf', 'hf', 'lf_hf', 'total_power']
# keep only columns that exist
candidate_cols = [c for c in candidate_cols if c in feats_df.columns]
print("Using numeric features:", candidate_cols)

feats_df_clean = feats_df.dropna(subset=['mean_hr', 'sdnn']).reset_index(drop=True)  # ensure core features present
# fill other nans with median
for c in candidate_cols:
    if feats_df_clean[c].isna().any():
        feats_df_clean[c] = feats_df_clean[c].fillna(feats_df_clean[c].median())

raw_windows_clean = np.array(raw_windows, dtype=object)
# align raw_windows to feats_df_clean indices (we appended in lock-step so slicing is okay)
if len(raw_windows_clean) != len(feats_df):
    # if differing, try to align using valid_mask
    valid_mask = feats_df[candidate_cols].notna().all(axis=1).values
    raw_windows_clean = raw_windows_clean[valid_mask]
else:
    # apply same dropna mask used above
    mask = feats_df.index[feats_df[['mean_hr', 'sdnn']].notna().all(axis=1)]
    raw_windows_clean = raw_windows_clean[mask]

print("Windows after cleaning:", feats_df_clean.shape[0], " raw_windows:", raw_windows_clean.shape[0])

# ---------------------------
# 6) Unsupervised clustering -> pseudo labels (k=3) same approach as your baseline
# ---------------------------
X = feats_df_clean[candidate_cols].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

k = 3
km = xgb.sklearn = None  # avoid name collision (we will import KMeans)
from sklearn.cluster import KMeans
km = KMeans(n_clusters=k, random_state=SEED, n_init=20)
labels = km.fit_predict(X_scaled)
sil = silhouette_score(X_scaled, labels)
print("Silhouette score (k=3):", round(sil, 3))

centers_df = pd.DataFrame(scaler.inverse_transform(km.cluster_centers_), columns=candidate_cols)
print("Cluster centers:\n", centers_df)

# map cluster -> pseudo_depth by SDNN (desc -> light=0)
order = centers_df.sort_values('sdnn', ascending=False).index.tolist()
cluster_to_depth = {c: i for i, c in enumerate(order)}
feats_df_clean['cluster'] = labels
feats_df_clean['pseudo_depth'] = feats_df_clean['cluster'].map(cluster_to_depth).astype(float)

# ---------------------------
# 7) Group-aware train/test split (preserve rec_index grouping)
# ---------------------------
groups = feats_df_clean['rec_index'].values
X_mat = feats_df_clean[candidate_cols].values
y = feats_df_clean['pseudo_depth'].values

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
train_idx, test_idx = next(gss.split(X_mat, y, groups))
X_train, X_test = X_mat[train_idx], X_mat[test_idx]
y_train, y_test = y[train_idx], y[test_idx]
groups_train = groups[train_idx]

print("Windows total:", len(y), "Train windows:", len(y_train), "Test windows:", len(y_test))

# scale using scaler fitted above (X_scaled corresponds to whole dataset)
X_train_scaled = scaler.transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# ---------------------------
# 8) Light random search of XGBoost hyperparameters (on training folds only)
# ---------------------------
def cv_mean_mae_for_params(params, Xtr, ytr, groups_tr, n_splits=4):
    # use GroupKFold on training records
    n_groups = len(np.unique(groups_tr))
    n_splits = min(n_splits, n_groups) if n_groups >= 2 else 2
    gkf = GroupKFold(n_splits=n_splits)
    maes = []
    for tr_idx, val_idx in gkf.split(Xtr, ytr, groups_tr):
        model = xgb.XGBRegressor(random_state=SEED, n_jobs=1, **params)
        model.fit(Xtr[tr_idx], ytr[tr_idx])
        preds = model.predict(Xtr[val_idx])
        maes.append(mean_absolute_error(ytr[val_idx], preds))
    return np.mean(maes)

# parameter search space (small, safe)
param_choices = {
    "max_depth": [3, 4, 5, 6],
    "learning_rate": [0.01, 0.03, 0.05, 0.1],
    "n_estimators": [100, 200, 400, 800],
    "subsample": [0.7, 0.8, 0.9, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "reg_alpha": [0.0, 1e-3, 1e-2],
    "reg_lambda": [1.0, 1.5, 2.0]
}

# sample a set of candidate configs randomly (safe count)
n_trials = 24
best_mae = 1e9
best_params = None
tried = 0
print("Starting small random search over XGBoost parameter space ({} trials)".format(n_trials))
for _ in range(n_trials):
    params = {
        "max_depth": random.choice(param_choices["max_depth"]),
        "learning_rate": random.choice(param_choices["learning_rate"]),
        "n_estimators": random.choice(param_choices["n_estimators"]),
        "subsample": random.choice(param_choices["subsample"]),
        "colsample_bytree": random.choice(param_choices["colsample_bytree"]),
        "reg_alpha": random.choice(param_choices["reg_alpha"]),
        "reg_lambda": random.choice(param_choices["reg_lambda"]),
    }
    try:
        mae_cv = cv_mean_mae_for_params(params, X_train_scaled, y_train, groups_train, n_splits=4)
    except Exception as e:
        print("Trial failed:", e)
        continue
    tried += 1
    if mae_cv < best_mae:
        best_mae = mae_cv
        best_params = params
    if tried % 6 == 0:
        print(f"  tried {tried} / {n_trials} - current best MAE (cv) = {best_mae:.4f}")
print("Random search finished. trials run:", tried)
print("Best CV MAE on train folds:", best_mae)
print("Best params:", best_params)

# if tuning failed for some reason, fall back to reasonable defaults
if best_params is None:
    best_params = {"max_depth":5, "learning_rate":0.05, "n_estimators":400, "subsample":0.9, "colsample_bytree":0.9, "reg_alpha":0.0, "reg_lambda":1.0}
    print("Using fallback params:", best_params)

# ---------------------------
# 9) Train final XGB on train set and evaluate on test set
# ---------------------------
final_model = xgb.XGBRegressor(random_state=SEED, n_jobs=1, **best_params)
final_model.fit(X_train_scaled, y_train, eval_set=[(X_test_scaled, y_test)], verbose=False)
test_preds = final_model.predict(X_test_scaled)
test_mae = mean_absolute_error(y_test, test_preds)
test_r2  = r2_score(y_test, test_preds)
print("[Final XGB] Test MAE: {:.4f} | R2: {:.4f}".format(test_mae, test_r2))

# feature importance (by gain)
try:
    imp = final_model.get_booster().get_score(importance_type='gain')
    # map back to column names
    fi = {candidate_cols[int(k.replace('f',''))]: v for k, v in imp.items() if k.startswith('f')}
    print("Top features by XGBoost gain (partial):")
    # sort descending
    for k,v in sorted(fi.items(), key=lambda x: -x[1])[:10]:
        print(f"  {k}: {v:.4f}")
except Exception:
    pass

# save artifacts
joblib.dump(final_model, os.path.join(OUT_DIR, "xgb_final_model.pkl"))
joblib.dump(scaler, os.path.join(OUT_DIR, "hrv_scaler.pkl"))
joblib.dump({"candidate_cols": candidate_cols, "best_params": best_params}, os.path.join(OUT_DIR, "meta.pkl"))
print("Saved artifacts to", OUT_DIR)


Loading parquet: /kaggle/input/physionet-ecg/scg_rhc_f16.parquet
Loaded parquet: /kaggle/input/physionet-ecg/scg_rhc_f16.parquet shape: (374, 20)
Using ECG column: ECG_lead_II
ECG sample counts per row — min/median/max: 7500 7500 7500


Records:   0%|          | 0/374 [00:00<?, ?it/s]

Extracted feature windows: (148, 14)
Using numeric features: ['mean_hr', 'sdnn', 'rmssd', 'pnn50', 'sd1', 'sd2', 'lf', 'hf', 'lf_hf', 'total_power']
Windows after cleaning: 148  raw_windows: 148
Silhouette score (k=3): 0.418
Cluster centers:
      mean_hr         sdnn        rmssd     pnn50          sd1          sd2  \
0  35.054123  1008.298631  1348.083816  0.905750   949.956762  1054.452711   
1  28.656391  1379.848841  1733.567958  0.953869  1222.582365  1476.074905   
2  41.217083   625.702350   854.070917  0.777838   602.627973   640.384940   

              lf             hf      lf_hf   total_power  
0  152012.988432  156448.221074   1.368202  5.030918e+05  
1  562998.273475  107734.185549  10.616380  1.022205e+06  
2   58508.085515   60523.649095   1.682025  1.788265e+05  
Windows total: 148 Train windows: 118 Test windows: 30
Starting small random search over XGBoost parameter space (24 trials)
  tried 6 / 24 - current best MAE (cv) = 0.1141
  tried 12 / 24 - current best MAE 